## Starting Example
Here we will load the UR10 manipulator and load it into the Meshcat visualiser with a desired configuration. You should see a new window open in your browser displaying the arm in its default state.

In [2]:
import casadi
import numpy as np

import pinocchio as pin
import pinocchio.casadi as cpin

import time

from example_robot_data import load

robot = load('ur10')

# Select the default configuration for the robot.
q0 = robot.q0
# OPTIONAL - Instead, choose a random configuration within the configuration space of the model
# q0 = pin.randomConfiguration(robot.model)

## visualise the robot
import meshcat
from pinocchio.visualize import MeshcatVisualizer
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
# Display the robot with the initial state q0
viz.display(q0)

# Display reference as a transparent red model
viz_ref = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz_ref.initViewer(viz.viewer)
viz_ref.loadViewerModel(rootNodeName="reference", color=[1.0, 0.0, 0.0, 0.1])

# viz_point =  MeshcatVisualizer()

# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


# Simulating the Dynamics Without Control
We will now use the rigid-body dynamics algorithms to forward-compute the dynamics of the robot from an initial state. When there is no joint stiffness or damping, the system will behave chaotically.

We are not accounting for collisions in this tutorial, as this would require a sophisticated physic simulator to handle the dynamics of contact, Meshcat is simply displaying the configuration of the robot.

In [31]:
# Control inputs to each joint of the robot (set to zero) 
u = np.zeros(robot.nq)

# This is a very basic forward-simulator of the dynamics of the system given a control law controller, where the dynamics are forward-stepped by a first-order approximation.
def simulate(T, dt, robot, viz, viz_ref, controller, fps = 30):
    frame_cnt = 0

    # nq - Dimension of the robots configuration vector q
    # nv - Dimension of the robots velocity and acceleration vectors (v, a)
    
    q = robot.q0 # Configuration vector q
    v = np.zeros(robot.nv) # Velocity vector v
    a = np.zeros(robot.nv) # Acceleration vector a

    # Determine number of steps to wait until next frame
    n_skip = int((1.0 / fps) / dt)

    t = 0
    while t <= T:
        # Compute controller for current instanct
        u = controller(robot, t, q, v)
        # Calculate the current acceleration of the robot given the configuration, velocity and input
        a = pin.aba(robot.model, robot.data, q, v, u)
        # Forward step
        v = v + dt * a
        q = q + dt * v
        
        t += dt
        
        # Display every 24 frames
        if frame_cnt % n_skip == 0:
            viz.display(q)
            viz_ref.display(robot.q0)
            time.sleep(1.0 / fps)
        
        frame_cnt += 1

Let's start with simulating the system with no control input

In [32]:
def null_control(robot, t, q, v):
    return np.zeros(robot.nv)

# Simulator step resolution (s)
dt = 0.001

viz.viewer.jupyter_cell()

# Simulate the system for 5 seconds
simulate(5.0, dt, robot, viz, viz_ref, null_control)

## PD Joint Control
To track a nominal state, we can introduce virtual spring-damping to each joint by using a PD controller to act at each joint proportional to the error and error rate at each time step, much like in Lecture 1. 

<b>Note :</b> We are not using integral control here for convenience.

In [31]:
def reference_generator(robot, t):
    qr = robot.q0
    # Provide sinusoidal generation to select joints
    qr[0] = np.sin(2.0 * t)
    qr[1] = -1.5 + 0.5 * np.sin(t)
    qr[2] = np.sin(t)
    qr[3] = 2.0 * np.sin(0.5 * t)
    
    vr = np.zeros(robot.nv)
    
    return (qr, vr)

# Control Law u = Kp (q - qr) + Kd (v - vr)
def pd_controller(robot, t, q, v):
    # Define a reference
    qr, vr = reference_generator(robot, t)
    # Define gains
    Kp = 20.0 * np.diag(np.ones(robot.nv))
    Kd = 1.0 * np.diag(np.ones(robot.nv))

    return  Kp.dot(qr - q) + Kd.dot(vr - v)

# Simulate the system for 5 seconds
simulate(5.0, dt, robot, viz, viz_ref, pd_controller)

## Computed Torque Control
That wasn't very good, was it? It looks as though our controller doesn't know the robot arm has weight, which is exactly what is happening. Let us instead view the controller from an acceleration perspective, if we can compute torques that achieve a desired acceleration in the joints, we can make this desired acceleration our error terms. We can then choose accelerations that are dynamically consistent with the robot, such that they also account for the dynamics of the robot.

In [32]:

def computed_torque_controller(robot, t, q, v):
    # Define a reference
    qr, vr = reference_generator(robot, t)
    ar = np.zeros(robot.nv)

    Kp = 25.0 * np.diag(np.ones(robot.nv))
    Kd = 2.0 * np.diag(np.ones(robot.nv))

    # Set desired acceleration of the system as the PD error of the state
    ad = ar + Kp.dot(qr - q) + Kd.dot(vr - v)
    # Compute control inputs that would realise this acceleration on the model, accounting for its dynamics
    u = pin.rnea(robot.model, robot.data, q, v, ad)
    
    return  u

# Simulate the system for 5 seconds
simulate(10.0, dt, robot, viz, viz_ref, computed_torque_controller)

As you can see, the dynamics are now accounted for in the controller, thus the robot hardly moves. It should be noted this is a perfect simulation, where we know the dynamics of the robot exactly, if the models differ significantly, the behaviour may grow unstable.

## Operational Space Control
Moving the arm to any arbitrary point in space can be difficult from a computed torque perspective, as we need to know what joint configuration is required. 

Operational space control allows us to specify tasks that the robot needs to perform, and computes a torque to achieve it. This can range from positional tasks and impedance control (specify a desired force or spring-damper-like behaviour) to more abstract goals such as stability criterion and centre of mass tracking.

Continuing with our Panda example, we will make the robot follow a specified trajectory in space using operational space control.

In [12]:
# Include symbolic algebra package Casadi
import casadi
from pinocchio import casadi as cpin
from scipy.optimize import fmin_bfgs

def task_space_reference_generator(robot, t):
    return (np.array([-0.5, 0.0, 0.6]), np.array([0.0, 0.0, 0.0]))


def operational_space_control(robot, t, q, v):
    # Compute the positions of all bodies in the robot by using forward kinematics
    pin.forwardKinematics(robot.model, robot.data, q)
    # Joint ID
    JOINT_ID = 3

    # Compute task-space acceleration
    M = pin.crba(robot.model, robot.data, q)
    h = pin.rnea(robot.model, robot.data, q, v, np.zeros(robot.nq))
    
    # Get the Jacobian of the position in space
    J = pin.computeJointJacobian(robot.model, robot.data, q, JOINT_ID)[3:6, :]
    # Get the time-rate of change of the derivative
    pin.computeJointJacobiansTimeVariation(robot.model, robot.data, q, v)
    dJ = pin.getJointJacobianTimeVariation(robot.model, robot.data, JOINT_ID, pin.LOCAL)[3:6, :]

    # These PD gains are now in task-space, this provides a further level of abstraction
    Kp = np.diag([3.0, 3.0, 3.0])
    Kd = np.diag([1.0, 1.0, 1.0])

    # Track the position of the tip of the robot
    xpos = robot.data.oMi[JOINT_ID].translation
    xvel = J.dot(q)

    # Task-space references
    xpos_r, xvel_r = task_space_reference_generator(robot, t)

    # Task space acceleration
    xacc = Kp.dot(xpos_r - xpos) + Kd.dot(xvel_r - xvel)
    
    # xacc = J qacc + dJ qvel
    qacc = np.linalg.pinv(J).dot(xacc - dJ.dot(v))
    
    # Compute the required torque
    u = pin.rnea(robot.model, robot.data, q, v, qacc)
    
    return u

# Display reference object

simulate(5.0, dt, robot, viz, viz_ref, operational_space_control)


## Model Predictive Control
Here we will generate an optimal control problem with a prediction horizon, using a linear estimate of the dynamics

In [26]:
from casadi import *
from scipy.optimize import fmin_bfgs

class ModelPredictiveControl:
    def __init__(self, N):
        self.N = N

    def setup(self, robot):
        # Create program
        cmodel = cpin.Model(robot.model)
        cdata = cmodel.createData()

        nq = robot.nq
        nv = robot.nv

        nx = nq + nv
        nu = nv
        ndx = 2 * nv

        # Compute the forward dynamics of the system

        # Compute the linearisation of the dynamics for MPC
        
        # Control horizon
        N = 10 

        # Use casadi's Opti solver interface to create a mathematical program
        self.opti = Opti()

        # ---- decision variables ---------
        # State trajectory
        X = self.opti.variable(nx, N + 1)        
        # Control input
        U = self.opti.variable(nu, N + 1)
        
        # Execution time
        T = self.opti.variable()
        
        # ---- parameters ---------
        X0 = self.opti.parameter(nx)

        # ---- objective ---------
        self.opti.minimize(T) # race in minimal time

        dt = T / N # length of a control interval
        
        # ---- dynamic constraints --------

        # Create the symbolic expression graph for the dynamics of the system, linearised about x0
        q = casadi.SX.sym('q', nq)
        v = casadi.SX.sym('v', nv)
        u = casadi.SX.sym('u', nu)
        q0 = casadi.SX.sym('q0', nq)
        v0 = casadi.SX.sym('v0', nq)

        x = casadi.vertcat(q, v)
        x0 = casadi.vertcat(q0, v0)

        

        f = casadi.vertcat(v, cpin.aba(cmodel, cdata, q0, v0, u))
        # Compute the jacobian of the system dynamics at the initial state X0
        A = casadi.jacobian(f, x0)
        B = casadi.jacobian(f, u)
        
        # Provide linearised model at the current state to estimate the next state
        xnext = casadi.mtimes(A, casadi.vertcat(q, v)) + casadi.mtimes(B, u)

        fun = casadi.Function(
            "f",
            [x, u, x0],
            [xnext]
        )

        for k in range(N): # loop over control intervalsd
            # First-order integration
            res = fun([X[:,k], U[k], X0])
            x_next = X[:,k] + dt * res
            self.opti.subject_to(X[:,k+1] == x_next)

        # ---- path constraints -----------
        # opti.subject_to(speed<=limit(pos))   # track speed limit
        # opti.subject_to(opti.bounded(0,U,1)) # control is limited

        # ---- boundary conditions --------
        self.opti.subject_to(X[0, :] == X0)   # Begin from initial state X0
        # self.opti.subject_to(pos[-1]==1)  # finish line at position 1

        # ---- misc. constraints  ----------
        self.opti.subject_to(T >= 0) # Time must be positive

        # ---- initial values for solver ---
        self.opti.set_initial(v, 1.0)
        self.opti.set_initial(T, 1)

    def solve(self, q, v, u):
        # ---- solve NLP              ------
        self.opti.solver("ipopt") # set numerical backend
        sol = self.opti.solve()   # actual solve


mpc = ModelPredictiveControl(10)
mpc.setup(robot)


NotImplementedError: Wrong number or type of arguments for overloaded function 'Function_call'.
  Possible prototypes are:
    call(self,dict:DM,bool,bool)
    call(self,[DM],bool,bool)
    call(self,[SX],bool,bool)
    call(self,dict:SX,bool,bool)
    call(self,dict:MX,bool,bool)
    call(self,[MX],bool,bool)
  You have: '(Function,([MX]))'
